In [ ]:
import pandas as pd
import numpy as np
import rasterio
import math
import geopandas as gpd
from shapely.geometry import Point
from fiona import Env
import matplotlib.pyplot as plt
from rasterio.mask import mask
import requests
import rioxarray
import os
from rasterio.mask import mask

from rasterio.transform import from_origin
from tqdm import tqdm
import mercantile
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, Resampling
from pyproj import CRS, Transformer
import subprocess
import sys
print(sys.path)

In [ ]:
# Get home directory
current_dir = os.getcwd()
homedir = os.path.dirname(current_dir)
print("Home Directory:", homedir)
input_dir = os.path.join(homedir, "Input") #this is the input directory  code
output_dir = os.path.join(homedir, "Output") #this is the output directory code

quadkey_path = os.path.join(homedir, "quadkey")
sys.path.append(quadkey_path)


import quadkey


def calculate_tile_bbox(row):
    qk = str(quadkey.from_geo((row['latitude'], row['longitude']), 14))
    b = mercantile.bounds(mercantile.quadkey_to_tile(qk))
    step_lon = abs(b.west - b.east)
    step_lat = abs(b.north - b.south)
    return step_lon, step_lat


def meters_to_longitude(meters, latitude):
    """ convert meters to decimal degrees, based on latitude
    World circumference measured around the equator is 40,075.017 km (source: wikipedia)
    40075.017 / 360 = 111.319491667 km/deg
    """
    return meters / (111.319491667 * 1000 * math.cos(latitude * (math.pi / 180)))


def meters_to_latitude(meters):
    """ convert meters to decimal degrees, based on latitude
    World circumference measured around the equator is 40,075.017 km (source: wikipedia)
    40075.017 / 360 = 111.319491667 km/deg
    """
    return meters / (111.132952778 * 1000)


def longitude_to_meters(degrees, latitude):
    """ convert decimal degrees to meters, based on latitude
    1 arc degree at the equator 111.132952778 km
    """
    return math.cos(latitude * (math.pi / 180)) * abs(degrees) * 111.319491667 * 1000


def latitude_to_meters(degrees):
    """ convert decimal degrees to meters, based on latitude
    World circumference measured around the equator is 40,007.863 km (source: wikipedia)
    40,007.863 / 360 = 111.132952778 km/deg
    """
    return abs(degrees) * 111.132952778 * 1000


def csv_to_raster(input_csv, output_raster, resolution=2445.98490512564):
    """ convert csv file to raster """
    # Read csv

    rwi_csv_path = os.path.join(input_dir, "ind_pak_relative_wealth_index.csv")
    df = pd.read_csv(rwi_csv_path)

    # Calculate raster cell size
    df['step_lon'], df['step_lat'] = zip(*df.apply(calculate_tile_bbox, axis=1))
    step_lon = df['step_lon'].mean()
    step_lat = df['step_lat'].mean()

    # Get minimum latitude and longitude
    lat_min = df.latitude.min()
    lon_min = df.longitude.min()
    # Get extent in longitude and latitude
    Dlat = abs(df.latitude.max() - lat_min)
    Dlon = abs(df.longitude.max() - lon_min)
    # Convert extent to the number of raster cells
    size_lat = round(Dlat / step_lat) + 1
    size_lon = round(Dlon / step_lon) + 1
    # Initialize empty array
    arr = np.empty((size_lat, size_lon))
    arr[:] = np.nan

    for ix, row in df.iterrows():
        dlat = abs(row['latitude'] - lat_min)
        dlon = abs(row['longitude'] - lon_min)
        ilat = round((size_lat - 1) * dlat / Dlat)
        ilon = round((size_lon - 1) * dlon / Dlon)
        # Correct for rasterio flip
        ilat = size_lat - ilat - 1
        arr[ilat, ilon] = row['rwi']

    # Transform array into raster
    transform = from_origin(lon_min - step_lon * 0.5,
                            df.latitude.max() + step_lat * 0.5,
                            step_lon, step_lat)
    if os.path.exists(output_raster):
        os.remove(output_raster)
    new_raster = rasterio.open(output_raster, 'w', driver='GTiff',
                               height=arr.shape[0], width=arr.shape[1],
                               count=1, dtype=str(arr.dtype),
                               crs='EPSG:4326',
                               transform=transform)
    # Save raster
    new_raster.write(arr, 1)
    new_raster.close()


def calculate_error(df_in, df_out):
    df_in = pd.read_csv(df_in)
    df_out = pd.read_csv(df_out)
    df_out.columns = ['longitude', 'latitude', 'rwi']

    df_out = df_out.dropna(subset=['rwi']).reset_index(drop=True)
    dist = pd.DataFrame()
    for ix, row in tqdm(df_in.iterrows(), total=df_in.shape[0]):
        lat = row['latitude']
        lon = row['longitude']
        df_out['dist'] = np.sqrt(np.power(df_out['latitude']-lat, 2) + np.power(df_out['longitude']-lon, 2))
        idxmin = df_out['dist'].idxmin()
        min_err_lat = abs(df_out.iloc[idxmin]['latitude'] - lat)
        min_err_lat_m = latitude_to_meters(min_err_lat)
        min_err_lon = abs(df_out.iloc[idxmin]['longitude'] - lon)
        min_err_lon_m = longitude_to_meters(min_err_lon, lat)

        dist = dist.append(pd.Series({'latitude': lat,
                                      'longitude': lon,
                                      'err_lat_deg': min_err_lat,
                                      'err_lat_met': min_err_lat_m,
                                      'err_lon_deg': min_err_lon,
                                      'err_lon_met': min_err_lon_m
                                      }), ignore_index=True)
    return dist

###clip the raster to pakistan's boundaries- using the unocha 2022 shapefile
def clip_raster(input_raster, output_raster, shapefile):
    """ Clip the raster using the provided shapefile """
    with rasterio.open(input_raster) as src:
        geoms = gpd.read_file(shapefile)
        geoms = geoms.to_crs(src.crs)
        geometries = geoms.geometry.values.tolist()
        out_image, out_transform = mask(src, geometries, crop=True)
        out_meta = src.meta.copy()

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)


if __name__ == "__main__":

    admin_input_dir = os.path.join(input_dir, "admin units")

    shapefile_path = os.path.join(admin_input_dir, "pak_admbnda_adm1_wfp_20220909.shp")

    if not os.path.exists(input_dir):  # Check if the input directory exists
        print(f"Input directory '{input_dir}' does not exist.")
        exit()


    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


    dist_all = pd.DataFrame()
    
    for file in os.listdir(input_dir):
        if file.endswith('.csv'):
            print(f"processing {file}")
            input_filepath = os.path.join(input_dir, file)
            output_filepath = os.path.join(output_dir, "RWI_Raster", file.replace('.csv', '.tif'))
            clipped_output_filepath = output_filepath.replace('ind_pak_relative_wealth_index.tif', 'PAK_relative_wealth_index.tif')

        
            csv_to_raster(input_filepath, output_filepath)
            clip_raster(output_filepath, clipped_output_filepath, shapefile_path)



In [ ]:
from osgeo import gdal

source_raster_path = os.path.join(output_dir, "RWI_Raster", "PAK_relative_wealth_index.tif")
target_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_SMOD.tif")
output_raster_path_ghs = os.path.join(output_dir, "RWI_Raster", "reprojected_to_match_GHS.tif")

# Open the source raster to get its CRS
with rasterio.open(source_raster_path) as src_source:
    # Open the target raster to get its CRS
    with rasterio.open(target_raster_path) as src_target:
        # Get the CRS of the target raster
        target_crs = src_target.crs

        # Reproject the source raster to match the CRS of the target raster
        transform, width, height = calculate_default_transform(
            src_source.crs, target_crs, src_source.width, src_source.height, *src_source.bounds
        )

        # Create the profile for the output raster
        output_profile = src_source.profile
        output_profile.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Open the output raster file for writing
        with rasterio.open(output_raster_path_ghs, 'w', **output_profile) as dst:
            # Reproject the data using nearest-neighbor resampling
            rasterio.warp.reproject(
                source=rasterio.band(src_source, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src_source.transform,
                src_crs=src_source.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )

###RESAMPLING-CHANGING RESOLUTION
# Open the reprojected raster

input_raster_path = os.path.join(output_dir, "RWI_Raster", "reprojected_to_match_GHS.tif")
pop_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_POP.tif")

input_raster = gdal.Open(input_raster_path)

# Set the desired resolution (1 km in this case)
target_resolution = (1000, 1000)  # 1 km resolution

# Resample the raster using bilinear interpolation (you can choose other methods)
output_raster_path = os.path.join(output_dir, "RWI_Raster", "RWI_resampled.tif")

gdal.Warp(output_raster_path, input_raster, xRes=target_resolution[0], yRes=target_resolution[1], resampleAlg=gdal.GRA_NearestNeighbour)


###CHECKING RESOLUTION
# Original RWI raster
rwi_raster_path = os.path.join(output_dir, "RWI_Raster", "RWI_resampled.tif")
pop_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_POP.tif")

with rasterio.open(rwi_raster_path) as src:
    print("Original RWI Raster:")
    print(f"  Bounds: {src.bounds}")
    print(f"  Resolution: {src.res}")
    print(f"  CRS: {src.crs}")

# Reprojected population raster
with rasterio.open(pop_raster_path) as src:
    print("Population Raster:")
    print(f"  Bounds: {src.bounds}")
    print(f"  Resolution: {src.res}")
    print(f"  CRS: {src.crs}")
    

import subprocess


# Set paths to your rasters
original_rwi_raster_path = os.path.join(output_dir, "RWI_Raster", "RWI_resampled.tif")
pop_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_POP.tif")
output_resampled_rwi_path = os.path.join(output_dir, "RWI_Raster", "RWI_reproj_resampled.tif")

# Open the population raster to get its bounds
population_dataset = gdal.Open(pop_raster_path)
target_extent = population_dataset.GetGeoTransform()

# Extract the target bounds
left, bottom, right, top = target_extent[0], target_extent[3] + (population_dataset.RasterYSize * target_extent[5]), target_extent[0] + (population_dataset.RasterXSize * target_extent[1]), target_extent[3]

# Build gdal_translate command
gdal_translate_cmd = f"gdal_translate -projwin {left} {top} {right} {bottom} -of GTiff {original_rwi_raster_path} {output_resampled_rwi_path}"

# Execute the gdal_translate command
subprocess.call(gdal_translate_cmd, shell=True)

# Close the dataset
population_dataset = None


####CHECKING RESOLUTION
# Original RWI raster
rwi_raster_path = os.path.join(output_dir, "RWI_Raster", "RWI_reproj_resampled.tif")
with rasterio.open(rwi_raster_path) as src:
    print("Original RWI Raster:")
    print(f"  Bounds: {src.bounds}")
    print(f"  Resolution: {src.res}")
    print(f"  CRS: {src.crs}")

# Reprojected population raster
pop_raster_path = os.path.join(input_dir, "GHSL_PAK_2020", "2020_GHS_PAK_POP.tif")
with rasterio.open(pop_raster_path) as src:
    print("Population Raster:")
    print(f"  Bounds: {src.bounds}")
    print(f"  Resolution: {src.res}")
    print(f"  CRS: {src.crs}")


In [ ]:

# After creating the resampled RWI raster and checking resolutions, you can delete the reprojected raster:
if os.path.exists(output_raster_path):
    os.remove(output_raster_path)
    print("Reprojected raster deleted successfully.")
else:
    print("Reprojected raster does not exist.")

